# Lib

In [76]:
%load_ext autoreload
%autoreload 2

import pandas as pd

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 10000)


import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sys
import os
import glob

if sys.platform=='win32':
    sys.path.insert(0,".\..\src")
        
elif sys.platform=='linux':
    sys.path.insert(0,"./../src")

from utils import utils_ml, parameters


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [77]:
if sys.platform=='win32':
    !dir .\
        
elif sys.platform=='linux':
    !ls ./

 O volume na unidade C � OS
 O N�mero de S�rie do Volume � C498-8BF5

 Pasta de c:\Users\caanp\OneDrive\Documents\repositories\ML-anti-fraud\notebooks

18/11/2022  17:50    <DIR>          .
18/11/2022  16:18    <DIR>          ..
10/09/2022  23:28    <DIR>          .ipynb_checkpoints
18/11/2022  16:18         1.047.962 1.EA.ipynb
18/11/2022  16:18           164.892 2.Feature_engineering.ipynb
18/11/2022  17:50           163.344 3.Modeling.ipynb
18/11/2022  16:34    <DIR>          catboost_info
               3 arquivo(s)      1.376.198 bytes
               4 pasta(s)   343.316.025.344 bytes dispon�veis


# Reading

In [78]:
## para saber en que directório uno se encuentra
#
if sys.platform=='win32':
    !echo %cd%
        
elif sys.platform=='linux':
    !pwd

c:\Users\caanp\OneDrive\Documents\repositories\ML-anti-fraud\notebooks


In [79]:
if sys.platform=='win32':
    df = pd.read_csv('.\..\data\dados.csv', sep=',', decimal=".")
        
elif sys.platform=='linux':
    df = pd.read_csv('./../data/dados.csv', sep=',', decimal=".")


## dropping column "o" becaise it has 78% missing. and this is a boolean columns
# df.drop("o", axis=1, inplace=True)

In [80]:
df

,a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,fecha,monto,score,fraude
0,4,0.7685,94436.24,20.0,0.444828,1.0,BR,5,Máquininha Corta Barba Cabelo Peito Perna Pelo...,cat_8d714cd,0.883598,240.0,102.0,1,NaN,N,2020-03-27 11:51:16,5.64,66,0
1,4,0.7550,9258.50,1.0,0.000000,33.0,BR,0,Avental Descartavel Manga Longa - 50 Un. Tnt ...,cat_64b574b,0.376019,4008.0,0.0,1,Y,N,2020-04-15 19:58:08,124.71,72,0
2,4,0.7455,242549.09,3.0,0.000000,19.0,AR,23,Bicicleta Mountain Fire Bird Rodado 29 Alumini...,cat_e9110c5,0.516368,1779.0,77.0,1,NaN,N,2020-03-25 18:13:38,339.32,95,0
3,4,0.7631,18923.90,50.0,0.482385,18.0,BR,23,Caneta Delineador Carimbo Olho Gatinho Longo 2...,cat_d06e653,0.154036,1704.0,1147.0,1,NaN,Y,2020-04-16 16:03:10,3.54,2,0
4,2,0.7315,5728.68,15.0,0.000000,1.0,BR,2,Resident Evil Operation Raccoon City Ps3,cat_6c4cfdc,0.855798,1025.0,150.0,1,NaN,N,2020-04-02 10:24:45,3.53,76,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149995,4,0.8191,21393.63,7.0,0.000000,1.0,BR,4,Aparelho Lipocavitação Ultrassônico + Gel Cond...,cat_a5b2091,0.808366,2009.0,434.0,1,NaN,N,2020-04-11 19:31:07,47.15,95,0
149996,4,NaN,NaN,24.0,0.384615,11.0,UY,20,"Sellos De Goma Automaticos, Personalizados.",cat_e39ab7e,0.989981,499.0,135.0,1,NaN,Y,2020-03-11 20:21:35,9.69,0,0
149997,4,NaN,NaN,3.0,0.477778,1.0,BR,19,Hélice 3 Pás Alumínio Rabeta 6.5 Hp Pesca Barc...,cat_ee6ecc8,0.763939,127.0,127.0,1,NaN,Y,2020-03-11 19:36:07,5.97,15,0
149998,4,0.6067,152906.86,1.0,0.099175,133.0,BR,3,Tela Display Lcd Galaxy J7 Neo J701 Com Brilho...,cat_237e2d0,0.382728,4373.0,123.0,1,NaN,Y,2020-03-16 07:13:24,25.83,59,0


### Renaming and formatting columns

In [81]:
df.rename(columns={"a":"a_int",
                    "p":"p_boolean","n":'n_boolean',
                     "k":"k_float","e":"e_float","h":"h_float","l":"l_float","f":"f_float","m":"m_float","m":"m_float","b":"b_float","c":"c_float","d":"d_float","o":"o_obj",
                     'i': 'produto','j':'categoria_produto','g':'pais'}
       , inplace = True)

## Transform categorical features
df['n_boolean'] = df['n_boolean'].astype("category")
df['p_boolean'] = df['p_boolean'].astype("category")


## creating columns os date and hour
df['data'] = df['fecha'].apply(lambda x: x[:10])
df['hora'] = df['fecha'].apply(lambda x: x[10:])

# WoE transformation

In [83]:
cats_cols = utils_ml.list_subset_words( df.columns.tolist() , ["bool","cat",'obj','pais'])
numeric_cols = [x for x in df.columns.tolist() if df[x].dtype.name!="object" and df[x].dtype.name!="category" and x!="fraude" ]

features = cats_cols + numeric_cols


features

['n_boolean',
 'p_boolean',
 'categoria_produto',
 'o_obj',
 'pais',
 'a_int',
 'b_float',
 'c_float',
 'd_float',
 'e_float',
 'f_float',
 'h_float',
 'k_float',
 'l_float',
 'm_float',
 'monto',
 'score']

### Encoding

In [84]:
dff = df.copy()

for variable in features:

    label = "fraude"
    dtype_ = 'categorical' if df[variable].dtype=='O' or df[variable].dtype=='category' else 'numerical'
    monotonic = True
    save_path_pkl = '.\..\src\\features\\'


    df = utils_ml.encode_woe_var(df, variable, label, dtype_, monotonic = True, save_path_pkl = save_path_pkl )

In [85]:
features_bins_woe = utils_ml.list_subset_words( df.columns.tolist(), ["bin"])
features_bins_woe

['n_boolean_bins',
 'p_boolean_bins',
 'categoria_produto_bins',
 'o_obj_bins',
 'pais_bins',
 'a_int_bins',
 'b_float_bins',
 'c_float_bins',
 'd_float_bins',
 'e_float_bins',
 'f_float_bins',
 'h_float_bins',
 'k_float_bins',
 'l_float_bins',
 'm_float_bins',
 'monto_bins',
 'score_bins']

In [86]:
len(features_bins_woe)

17

# Saving

In [87]:
df

,a_int,b_float,c_float,d_float,e_float,f_float,pais,h_float,produto,categoria_produto,k_float,l_float,m_float,n_boolean,o_obj,p_boolean,fecha,monto,score,fraude,data,hora,n_boolean_bins,n_boolean_cat,n_boolean_WoE,p_boolean_bins,p_boolean_cat,p_boolean_WoE,categoria_produto_bins,categoria_produto_cat,categoria_produto_WoE,o_obj_bins,o_obj_cat,o_obj_WoE,pais_bins,pais_cat,pais_WoE,a_int_bins,a_int_cat,a_int_WoE,b_float_bins,b_float_cat,b_float_WoE,c_float_bins,c_float_cat,c_float_WoE,d_float_bins,d_float_cat,d_float_WoE,e_float_bins,e_float_cat,e_float_WoE,f_float_bins,f_float_cat,f_float_WoE,h_float_bins,h_float_cat,h_float_WoE,k_float_bins,k_float_cat,k_float_WoE,l_float_bins,l_float_cat,l_float_WoE,m_float_bins,m_float_cat,m_float_WoE,monto_bins,monto_cat,monto_WoE,score_bins,score_cat,score_WoE
0,4,0.7685,94436.24,20.0,0.444828,1.0,BR,5,Máquininha Corta Barba Cabelo Peito Perna Pelo...,cat_8d714cd,0.883598,240.0,102.0,1,NaN,N,2020-03-27 11:51:16,5.64,66,0,2020-03-27,11:51:16,0,[1],-0.28768,1,['N'],0.445779,8,['cat_9a9a0f6' 'cat_05f0709' 'cat_b50d9ab' 'ca...,0.371886,-1,Missing,0.919696,1,['BR' 'ES' 'PT' 'RU' 'CL' 'FR' 'NZ' 'DE' 'EC' ...,0.106995,1,"[3.50, inf)",-0.120984,4,"[0.75, 0.81)",-0.130586,1,"[649.68, inf)",-0.079304,4,"[19.50, 43.50)",-0.229870,2,"[0.18, inf)",-0.189859,1,"[0.50, 1.50)",0.312697,1,"[0.50, 5.50)",0.049907,1,"[0.12, 0.92)",0.003278,1,"[140.50, 339.50)",0.619954,3,"[49.50, 153.50)",-0.019221,1,"[5.24, 13.68)",-0.113703,1,"[65.50, 71.50)",-0.240343
1,4,0.7550,9258.50,1.0,0.000000,33.0,BR,0,Avental Descartavel Manga Longa - 50 Un. Tnt ...,cat_64b574b,0.376019,4008.0,0.0,1,Y,N,2020-04-15 19:58:08,124.71,72,0,2020-04-15,19:58:08,0,[1],-0.28768,1,['N'],0.445779,2,['cat_16839f1' 'cat_7da1387' 'cat_7f669a9' 'ca...,-0.642311,0,['Y'],0.270113,1,['BR' 'ES' 'PT' 'RU' 'CL' 'FR' 'NZ' 'DE' 'EC' ...,0.106995,1,"[3.50, inf)",-0.120984,4,"[0.75, 0.81)",-0.130586,1,"[649.68, inf)",-0.079304,0,"(-inf, 1.50)",0.609339,0,"(-inf, 0.07)",0.167535,7,"[21.50, 41.50)",-0.806365,0,"(-inf, 0.50)",0.629282,1,"[0.12, 0.92)",0.003278,10,"[3572.50, 5512.50)",-0.755314,0,"(-inf, 4.50)",0.910357,4,"[55.14, 147.69)",0.224128,2,"[71.50, 81.50)",0.195818
2,4,0.7455,242549.09,3.0,0.000000,19.0,AR,23,Bicicleta Mountain Fire Bird Rodado 29 Alumini...,cat_e9110c5,0.516368,1779.0,77.0,1,NaN,N,2020-03-25 18:13:38,339.32,95,0,2020-03-25,18:13:38,0,[1],-0.28768,1,['N'],0.445779,7,['cat_630e30e' 'cat_cf03adb' 'cat_d2b7673' 'ca...,0.171850,-1,Missing,0.919696,0,['AD' 'GH' 'GR' 'IL' 'IN' 'IT' 'JP' 'LB' 'NI' ...,-0.397711,1,"[3.50, inf)",-0.120984,3,"[0.72, 0.75)",-0.149816,1,"[649.68, inf)",-0.079304,2,"[2.50, 8.50)",0.067048,0,"(-inf, 0.07)",0.167535,6,"[12.50, 21.50)",-0.519325,4,"[19.50, 23.50)",-0.156024,1,"[0.12, 0.92)",0.003278,7,"[1677.50, 2085.50)",-0.249283,3,"[49.50, 153.50)",-0.019221,5,"[147.69, inf)",0.853462,5,"[92.50, inf)",1.837865
3,4,0.7631,18923.90,50.0,0.482385,18.0,BR,23,Caneta Delineador Carimbo Olho Gatinho Longo 2...,cat_d06e653,0.154036,1704.0,1147.0,1,NaN,Y,2020-04-16 16:03:10,3.54,2,0,2020-04-16,16:03:10,0,[1],-0.28768,0,['Y'],-0.562218,8,['cat_9a9a0f6' 'cat_05f0709' 'cat_b50d9ab' 'ca...,0.371886,-1,Missing,0.919696,1,['BR' 'ES' 'PT' 'RU' 'CL' 'FR' 'NZ' 'DE' 'EC' ...,0.106995,1,"[3.50, inf)",-0.120984,4,"[0.75, 0.81)",-0.130586,1,"[649.68, inf)",-0.079304,5,"[43.50, inf)",-0.588075,2,"[0.18, inf)",-0.189859,6,"[12.50, 21.50)",-0.519325,4,"[19.50, 23.50)",-0.156024,1,"[0.12, 0.92)",0.003278,7,"[1677.50, 2085.50)",-0.249283,9,"[965.50, inf)",-1.043734,0,"(-inf, 5.24)",-0.534456,0,"(-inf, 65.50)",-0.919400
4,2,0.7315,5728.68,15.0,0.000000,1.0,BR,2,Resident Evil Operation Raccoon City Ps3,cat_6c4cfdc,0.855798,1025.0,150.0,1,NaN,N,2020-04-02 10:24:45,3.53,76,0,2020-04-02,10:24:45,0,[1],-0.28768,1,['N'],0.445779,6,['cat_e90d7a7' 'cat_d016d08' 'cat_ecc6934' 'ca...,0.010108,-1,Missing,0.919696,1,['BR' 'ES' 'PT' 'RU' 'CL' 'FR' 'NZ' 'DE' 'EC' ...,0.106995,0,"(-inf, 3.50)",0.537906,3,"[0.72, 0.75)",-0.149816,1,"[649.68, inf)",-0.079

In [95]:
df.to_parquet(".\\..\data\dados_transformed.parquet", index=False)

# Requirements

In [96]:
path_root = parameters.root
!pip list --format=freeze > $path_root/requirements.txt